# This notebook explores the figure 3

In [13]:
%load_ext autoreload
%autoreload 2

from convsep.dataset import LargeDatasetMulti
from trainDNN import build_ca, train_auto
from phase_transform import PhaseTransform
from scipy.signal import blackmanharris, gaussian
import os
import theano.tensor as T
from theano.sandbox.rng_mrg import MRG_RandomStreams as RandomStreams
import theano
import lasagne
from lasagne.layers import ReshapeLayer,Layer
from lasagne.init import Normal
from lasagne.regularization import regularize_layer_params_weighted, l2, l1
from lasagne.regularization import regularize_layer_params
import convsep.util as util
import numpy as np
import time


ld2 = LargeDatasetMulti(path_transform_in="results/features", nsources=4,
                    batch_size=16, batch_memory=200,
                    time_context=11, overlap=75, nprocs=4,
                    mult_factor_in=0.3, mult_factor_out=0.3,
                    extra_features=True, model="p")
ld2.extra_feat_size = 2049


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
I 2019-03-11 15:53:16 dataset:1127 found 2 files
I 2019-03-11 15:53:20 dataset:1141 iteration size 21


In [14]:
db = "../DSD100subset/"
model="dsd_fft_1024"
nepochs = 40
scale_factor = 0.3
tt = PhaseTransform(frameSize=4096, hopSize=1024, sampleRate=44100, window=gaussian, std=0.4)

# train_errs = train_auto(train=ld2, fun=build_ca, transform=tt,
#                 outdir=os.path.join(db,'output',model),
#                 testdir=os.path.join(db,'Mixtures'),
#                 model=os.path.join(db,'models',"model_"+model+".pkl"),
#                 num_epochs=nepochs,
#                 scale_factor=scale_factor)

fun = build_ca
train = ld2
load=False
skip_train=False
skip_sep=False
chunk_size=60
chunk_overlap=2
nsamples=40
batch_size=32
batch_memory=50
time_context=30
overlap=25
nprocs=4
mult_factor_in=0.3
mult_factor_out=0.3
transform = tt
outdir=os.path.join(db,'output',model)
testdir=os.path.join(db,'Mixtures')
model=os.path.join(db,'models',"model_"+model+".pkl")
num_epochs=nepochs
scale_factor=scale_factor


amp = T.tensor4('amp')
dt_ph = T.tensor4('dt_ph')
df_ph = T.tensor4('df_ph')
# input_var2 = T.tensor4('inputs')
input_var = T.tensor4('inputs')
input_mask = T.tensor4('input_mask')
target_var = T.tensor4('targets')

theano_rng = RandomStreams(128)

eps=1e-12

sources = ['vocals','bass','drums','other']

nchannels = int(train.channels_in)
nsources = int(train.channels_out/train.channels_in)

print('nchannels: ', nchannels)
print('nsources: ', nsources)

input_size = int(float(transform.frameSize) / 2 + 1)

rand_num = theano_rng.normal( size = (batch_size,nsources,time_context,input_size), avg = 0.0, std = 0.1, dtype=theano.config.floatX)

nchannels:  2
nsources:  4


In [15]:
network = fun(amp=amp, df_ph=df_ph, dt_ph=dt_ph,
        batch_size=batch_size,time_context=time_context,
        feat_size=input_size, nchannels=nchannels, nsources=nsources)

if load:
    params=load_model(model)
    lasagne.layers.set_all_param_values(network,params)

prediction = lasagne.layers.get_output(network, deterministic=True)

sourceall=[]
errors_insts = []
loss = 0

sep_chann = []

loss = lasagne.objectives.squared_error(prediction, abs(target_var[:,:,5,:]))
loss = loss.mean()

MemoryError: 

In [ ]:
import time 

params1 = lasagne.layers.get_all_params(network, trainable=True)
updates = lasagne.updates.adadelta(loss, params1)

losser=[]

train_fn = theano.function([amp, df_ph, dt_ph, target_var], loss, updates=updates)

In [ ]:
theano.config.exception_verbosity = "high"

if not skip_train:
#     for epoch in range(num_epochs):
#         train_err = 0
#         train_batches = 0
#         errs = np.zeros((nchannels,nsources))
#         start_time = time.time()
#         for batch in range(train.iteration_size):
#             mag, target, features = train()
#             df_ph, dt_ph = features[..., :]
#             train_err += train_fn_mse(mag, target)
#             errs += np.array(train_fn1(mag, df_ph, dt_ph, target))
#             train_batches += 1

#         logging.info("Epoch {} of {} took {:.3f}s".format(
#             epoch + 1, num_epochs, time.time() - start_time))
#         logging.info("  training loss:\t\t{:.6f}".format(train_err/train_batches))
#         for j in range(nchannels):
#             for i in range(nsources):
#                 logging.info("  training loss for "+sources[i]+" in mic "+str(j)+":\t\t{:.6f}".format(errs[j][i]/train_batches))

#         model_noILD = model[:-4] + '_noILD' + model[-4:]
#         print('model_noILD: ', model_noILD)
#         save_model(model_noILD,network)
#         losser.append(train_err/train_batches)
        
        
    for epoch in range(num_epochs):
        # In each epoch, we do a full pass over the training data:
        train_err = 0
        train_batches = 0
        start_time = time.time()
        errs = np.zeros((nchannels,nsources))
        for batch in range(train.iteration_size):
            mag, target, features = train()
            df_ph = features[..., 0]
            dt_ph = features[..., 1]
            errs += np.array(train_fn(mag, df_ph, dt_ph, target))
            train_batches += 1


        # And a full pass over the validation data:
#         val_err = 0
#         val_acc = 0
#         val_batches = 0
#         for batch in iterate_minibatches(X_val, y_val, 500, shuffle=False):
#             inputs, targets = batch
#             err, acc = val_fn(inputs, targets)
#             val_err += err
#             val_acc += acc
#             val_batches += 1

        # Then we print the results for this epoch:
        print("Epoch {} of {} took {:.3f}s".format(
            epoch + 1, num_epochs, time.time() - start_time))
        print("  training loss:\t\t{:.6f}".format(err / train_batches))
#         print("  training loss:\t\t{:.6f}".format(train_err / train_batches))
#         print("  validation loss:\t\t{:.6f}".format(val_err / val_batches))
#         print("  validation accuracy:\t\t{:.2f} %".format(
#             val_acc / val_batches * 100))

In [14]:
inputs, targets, features = train()

In [10]:
train_fn(mag, df_ph, dt_ph, target)

ValueError: Shape mismatch: x has 45078 cols (and 16 rows) but y has 122940 rows (and 500 cols)
Apply node that caused the error: Dot22(Reshape{2}.0, W)
Toposort index: 45
Inputs types: [TensorType(float64, matrix), TensorType(float64, matrix)]
Inputs shapes: [(16, 45078), (122940, 500)]
Inputs strides: [(360624, 8), (4000, 8)]
Inputs values: ['not shown', 'not shown']
Outputs clients: [[Elemwise{Add}[(0, 0)](Dot22.0, InplaceDimShuffle{x,0}.0)]]

Backtrace when the node is created(use Theano flag traceback.limit=N to make it longer):
  File "C:\ProgramData\Anaconda3\envs\py36\lib\site-packages\IPython\core\interactiveshell.py", line 2845, in _run_cell
    return runner(coro)
  File "C:\ProgramData\Anaconda3\envs\py36\lib\site-packages\IPython\core\async_helpers.py", line 67, in _pseudo_sync_runner
    coro.send(None)
  File "C:\ProgramData\Anaconda3\envs\py36\lib\site-packages\IPython\core\interactiveshell.py", line 3020, in run_cell_async
    interactivity=interactivity, compiler=comp